In [112]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import interpolate
from scipy.interpolate import InterpolatedUnivariateSpline

In [113]:
data = pd.read_excel(r'/Users/shakhawathossainturag/Downloads/HT_NT_Data.xlsx')

# industrie_prices_without_VAT = pd.read_excel(r'/Users/shakhawathossainturag/Downloads/HT_NT_Data.xlsx",sheet_name='5.8.3 Strom - € - Industrie', skiprows = 5, nrows = 26, index_col = 0)
data.head()

,Hour of the Day,Price (Euro/MWh),Unnamed: 2,Mean HT (Euro/MWh),Unnamed: 4,dtDate,intHour,dblPrice,Mean Yearly Price (Euro/MWh),dtDate.1,...,Mean HT,dtDate.2,intHour.2,dblPrice.2,Mean NT,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Haupt Traif (HT): 8:00-19:00
0,8.0,27.21,NaN,32.119344,Wednesday,2020-01-01,1,41.88,30.471694,2020-01-01,...,NaN,2020-01-01,1,41.88,NaN,NaN,NaN,NaN,NaN,Neben Tarif (NT): 20:00-7:00 + Weekends
1,9.0,27.96,NaN,NaN,NaN,2020-01-01,2,38.60,NaN,2020-01-01,...,NaN,2020-01-01,2,38.60,NaN,NaN,NaN,NaN,NaN,NaN
2,10.0,25.50,NaN,NaN,NaN,2020-01-01,3,36.55,NaN,2020-01-01,...,NaN,2020-01-01,3,36.55,NaN,NaN,NaN,NaN,NaN,NaN
3,11.0,21.97,NaN,NaN,NaN,2020-01-01,4,32.32,NaN,2020-01-01,...,NaN,2020-01-01,4,32.32,NaN,NaN,NaN,NaN,NaN,NaN
4,12.0,20.27,NaN,NaN,NaN,2020-01-01,5,30.85,NaN,2020-01-01,...,NaN,2020-01-01,5,30.85,NaN,NaN,NaN,NaN,NaN,NaN


In [114]:
new_data = data[["dtDate.2", "intHour.2", "dblPrice.2"]]
new_data = new_data.rename(columns={"dtDate.2": "Date", "intHour.2": "hour", "dblPrice.2": "price"})
new_data.head()

,Date,hour,price
0,2020-01-01,1,41.88
1,2020-01-01,2,38.60
2,2020-01-01,3,36.55
3,2020-01-01,4,32.32
4,2020-01-01,5,30.85


In [122]:
# https://stackoverflow.com/questions/47150709/how-to-create-a-calendar-table-date-dimension-in-pandas

def create_date_table(start='2020-01-01', end='2020-12-31'):
    start_ts = pd.to_datetime(start).date()

    end_ts = pd.to_datetime(end).date()

    # record timetsamp is empty for now
    dates =  pd.DataFrame(columns=['Record_timestamp'],
        index=pd.date_range(start_ts, end_ts))
    dates.index.name = 'Date'

    days_names = {
        i: name
        for i, name
        in enumerate(['Monday', 'Tuesday', 'Wednesday',
                      'Thursday', 'Friday', 'Saturday', 
                      'Sunday'])
    }

    dates['Day'] = dates.index.dayofweek.map(days_names.get)
    dates['Month'] = dates.index.month
    dates.reset_index(inplace=True)
    dates.index.name = 'date_id'
    return dates

In [123]:
df = create_date_table()
df = df[["Date", "Day"]]
df

,Date,Day
date_id,,
0,2020-01-01,Wednesday
1,2020-01-02,Thursday
2,2020-01-03,Friday
3,2020-01-04,Saturday
4,2020-01-05,Sunday
...,...,...
361,2020-12-27,Sunday
362,2020-12-28,Monday
363,2020-12-29,Tuesday


In [124]:
df_with_data = pd.merge(new_data, df, how="inner", on="Date")
df_with_data

,Date,hour,price,Day
0,2020-01-01,1,41.88,Wednesday
1,2020-01-01,2,38.60,Wednesday
2,2020-01-01,3,36.55,Wednesday
3,2020-01-01,4,32.32,Wednesday
4,2020-01-01,5,30.85,Wednesday
...,...,...,...,...
8779,2020-12-31,20,59.47,Thursday
8780,2020-12-31,21,56.70,Thursday
8781,2020-12-31,22,52.44,Thursday
8782,2020-12-31,23,51.86,Thursday


In [125]:
df_with_data.Day.unique()

array(['Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday', 'Monday',
       'Tuesday'], dtype=object)

In [126]:
yearly_mean = df_with_data.price.mean()

In [127]:
#haupt_tarrif = df_with_data
haupt_tarrif = df_with_data[df_with_data["hour"].isin([8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]) & df_with_data["Day"].isin(['Wednesday', 'Thursday', 'Friday', 'Monday', 'Tuesday'])]
haupt_tarrif.head()

,Date,hour,price,Day
7,2020-01-01,8,30.00,Wednesday
8,2020-01-01,9,30.65,Wednesday
9,2020-01-01,10,30.65,Wednesday
10,2020-01-01,11,30.27,Wednesday
11,2020-01-01,12,30.34,Wednesday


In [128]:
haupt_tarrif.Day.unique()

array(['Wednesday', 'Thursday', 'Friday', 'Monday', 'Tuesday'],
      dtype=object)

In [129]:
haupt_tarrif.Day.count()

3144

In [130]:
df = 70 rows
HT = 25 
NT = 70 -25 = 45

SyntaxError: invalid syntax (2203680852.py, line 1)

In [131]:
cond = df_with_data['hour'].isin(haupt_tarrif['hour'])
df_with_data.drop(haupt_tarrif[cond].index, inplace = True)


/var/folders/3d/5tmbtv_x705103j1f7_ct0dw0000gn/T/ipykernel_19993/452124137.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_with_data.drop(haupt_tarrif[cond].index, inplace = True)


In [132]:
#neben_tarrif = df_with_data[(df_with_data["hour"].isin([1, 2, 3, 4, 5, 6, 7, 20, 21, 22, 23, 24]) & df_with_data["Day"].isin(['Wednesday', 'Thursday', 'Friday', 'Monday', 'Tuesday'])) |(df_with_data["Day"].isin(['Saturday', 'Sunday']))]

#neben_tarrif.head()

In [144]:
haupt_tarrif.head()

,Date,hour,price,Day
7,2020-01-01,8,30.00,Wednesday
8,2020-01-01,9,30.65,Wednesday
9,2020-01-01,10,30.65,Wednesday
10,2020-01-01,11,30.27,Wednesday
11,2020-01-01,12,30.34,Wednesday


In [145]:
neben_tarrif = df_with_data
neben_tarrif.head()

,Date,hour,price,Day
0,2020-01-01,1,41.88,Wednesday
1,2020-01-01,2,38.60,Wednesday
2,2020-01-01,3,36.55,Wednesday
3,2020-01-01,4,32.32,Wednesday
4,2020-01-01,5,30.85,Wednesday


In [134]:
neben_tarrif.count()

Date     5640
hour     5640
price    5640
Day      5640
dtype: int64

In [135]:
5640 + 3144

8784

In [136]:
haupt_tarrif.price.mean()

37.01998727735369

In [137]:
neben_tarrif.price.mean()

26.82136879432624

In [139]:
yearly_mean

30.471693989071035

In [140]:
ht_factor = haupt_tarrif.price.mean()/yearly_mean
ht_factor

1.2148975797220616

In [141]:
nt_factor = neben_tarrif.price.mean()/yearly_mean
nt_factor

0.8802060300272765